In [1]:
from ddeserts.annotate import prop_ests
from ddeserts.annotate import prop_moes
from ddeserts.annotate import sum_moes
from ddeserts.load import load_age_sex_cit_data
from ddeserts.load import load_felon_disf_data
from ddeserts.tsp import PROP_PRISON_NON_CIT

In [2]:
pop_df = load_age_sex_cit_data()
felon_df = load_felon_disf_data()

In [3]:
adu_est = pop_df['adu_est'].sum()
cvap_est = pop_df['cvap_est'].sum()

prop_adu_cit = cvap_est / adu_est
prop_adu_non_cit = 1 - prop_adu_cit
prop_adu_non_cit


0.07498392290080158

In [4]:
prop_prison_cit = 1 - PROP_PRISON_NON_CIT

# rate of (adult) citizens being in prison, relative to their proportion
# of the adult population
cit_prison_rate = prop_prison_cit / prop_adu_cit
# same, for non-citizens
non_cit_prison_rate = PROP_PRISON_NON_CIT / prop_adu_non_cit

# estimated rate at which adult non-citizens are in prison,
# relative to citizens
non_cit_prison_ratio = non_cit_prison_rate / cit_prison_rate
non_cit_prison_ratio

0.6356187305556319

In [5]:
df = pop_df.merge(felon_df, on=['geoname', 'geotype'])

In [9]:
df['ncvap_est'] = (df['adu_est'] - df['cvap_est']).clip(0)
df['ncvap_moe'] = sum_moes(df, 'adu', 'cvap')
df['prop_adu_non_cit_est'] = prop_ests(df, 'ncvap', 'adu')
df['prop_adu_non_cit_moe'] = prop_moes(df, 'ncvap', 'adu')


In [13]:
df['prop_adu_felon_disf_est'] = prop_ests(df, 'felon_disf', 'adu')

In [16]:
# estimate disenfranchised adult population
# (overestimate, doesn't account for non-cit felons)

df['prop_adu_dvap_est'] = df['prop_adu_non_cit_est'] + df['prop_adu_felon_disf_est']

In [19]:
df.sort_values('prop_adu_dvap_est')[['geoname', 'prop_adu_non_cit_est', 'prop_adu_felon_disf_est', 'prop_adu_dvap_est']]

,geoname,prop_adu_non_cit_est,prop_adu_felon_disf_est,prop_adu_dvap_est
44,Vermont,0.015992,0.000000,0.015992
18,Maine,0.017666,0.000000,0.017666
25,Montana,0.013163,0.004752,0.017915
47,West Virginia,0.009014,0.009985,0.019000
28,New Hampshire,0.026283,0.002207,0.028491
34,Ohio,0.025608,0.005111,0.030719
33,North Dakota,0.031585,0.002594,0.034179
40,South Dakota,0.018454,0.019488,0.037942
21,Michigan,0.033640,0.004450,0.038090
24,Missouri,0.022677,0.017192,0.039869
